In [0]:
from urllib.request import urlopen,Request
import requests
import re
import xlwt
from html import unescape


def clean(value):
  value = re.sub('<br />',' ',value)
  value = unescape(value)
  value = re.sub('<[^>]*?>','',value)
  return value

def my_regex(regex,content):
  result = re.findall(regex,content)
  print('result::',result)
  if result:
    result = result[0]
  else:
    result = ''
  return result

website = 'https://www.powtech.de/en/ausstellerprodukte/exhibitorlist?zip=&catids=31408358%2C31408529%2C31408578%2C31408645%2C31408775%2C31408341%2C31408342%2C31408343%2C31408344%2C31408345%2C31408346&halls=&countries=&edbid=&filterchar=*&items=&search=&tab=1'
head = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.88 Safari/537.36','cookie': 'ASP.NET_SessionId=jppp4wuspzijufp2r3l2rhat'}
while True:
  req = Request(website, headers=head)
  mainContent = unescape(urlopen(req).read().decode('UTF-8'))
  #print('mainContent:',mainContent)
  with open ('data.html','w') as f:
    f.write(mainContent)

  mainBlock = re.findall('(<div\s*class\=\"rcEntry\">[\w\W]*?</span>\s*<div\s*class\=\"ePosition\"[\w\W]*?</a>\s*</div>)',mainContent)
  print(len(mainBlock))

  for block in mainBlock:
    title = my_regex('<div\s*class\=\"eInfo\"\>[^>]*?><a\s*href\=\"[^>]*?>([^>]*?)</a></h2>\s*<span\s*class\=\"exhibitor[^>]*?>([^>]*?)</span>',block)
    print('title:',title)
    sublink = re.findall('<a\s*href\=\"([^>]*?)\"',block)
    subURL = 'https://www.powtech.de' + sublink[0]
    print('subURL:',subURL)
    input('========================================')
    subreq = Request(subURL, headers=head)
    subContent = unescape(urlopen(subreq).read().decode('UTF-8'))
    #print('subContent:',subContent)
    #with open('subdata.html','w') as h:
      #h.write(subContent)
    street = my_regex('<span\s*itemprop\=\"streetAddress\">([^>]*?)</span>',subContent)
    post_code = my_regex('span\s*itemprop\=\"postalCode\">([^>]*?)</span>',subContent)
    locality = my_regex('<span\s*itemprop\=\"addressLocality\">([^>]*?)</sp',subContent)
    country = my_regex('<span\s*itemprop\=\"addressCountry\">([^>]*?)</span>',subContent)
    location = str(street) +' | ' +str(post_code)  +' | ' + str(locality)  +' | ' + str(country)
    print('location::',location)
    phone = my_regex('<span\s*itemprop\=\"telephone\">([^>]*?)</span>',subContent)
    fax = my_regex('<span\s*itemprop\=\"faxNumber\">([^>]*?)</span>',subContent)
    
  nextPageLink = re.findall('<a\s*href\=\"([^>]*?)\"\s*class\=\"last\">',mainContent)
  if nextPageLink:
    nextPageLink = 'https://www.powtech.de' + nextPageLink[0]
    website = nextPageLink
    print("nextPageLink:",nextPageLink)
  else:
    print('No NextPage.')
    break
  input('=====')
    
      
      
	
	

